<a href="https://colab.research.google.com/github/Perisique/try/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np
import re
import string
import spacy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer,  TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
!python3 -m spacy download en_core_web_sm

2023-10-17 09:22:03.720594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Загружаем датасет:

In [45]:
data = pd.read_csv('spam_or_not_spam.csv', encoding='iso-8859-1')
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


Токенизация

In [92]:
def custom_tokenize(text):
  text = re.sub(r'[^a-zA-Z ]', '', text)
  return text.split()
custom_vectorizer = CountVectorizer(
    max_df=0.7,
    min_df=0.005,
    tokenizer=custom_tokenize,
)

Делим на тестовую и тренировочную выборки

In [125]:
X_train, X_test, y_train, y_test = train_test_split(data['email'], data['label'], random_state=500)
vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
X_train_vectorized = custom_vectorizer.fit_transform(X_train.values.astype('U'))
X_test_vectorized = custom_vectorizer.transform(X_test.values.astype('U'))
pd.DataFrame(X_train_vectorized.toarray(), columns=custom_vectorizer.get_feature_names_out()).head()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ability,able,about,above,absolute,absolutely,abuse,accept,acceptable,acceptance,...,york,you,young,your,yours,yourself,z,zero,zip,zzzz
0,0,0,2,0,0,0,0,1,0,0,...,0,2,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
X_train, X_test, y_train, y_test = train_test_split(data['email'], data['label'], random_state=2023)
vectorizer = CountVectorizer(max_df=0.7, min_df=0.003) # max_df фильтрует corpus-specific stop words
X_train_vectorized = vectorizer.fit_transform(X_train.values.astype('U'))
X_test_vectorized = vectorizer.transform(X_test)

In [129]:
logreg = LogisticRegression().fit(X_train_vectorized, y_train)
preds = logreg.predict(X_test_vectorized)
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train.values.astype('U'), y_train)
preds1 = pipe.predict(X_test.values.astype('U'))
print(classification_report(y_test, preds))
print(classification_report(y_test, preds1))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       616
           1       0.98      0.93      0.95       134

    accuracy                           0.98       750
   macro avg       0.98      0.96      0.97       750
weighted avg       0.98      0.98      0.98       750

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       616
           1       1.00      0.81      0.89       134

    accuracy                           0.97       750
   macro avg       0.98      0.90      0.94       750
weighted avg       0.97      0.97      0.96       750



Большая точность получилась у CountVectorizer, однако tfidfVectorizer лучше нашёл весь спам. Далее работаем с CountVectorizer.

In [123]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
print(clf.get_params())
clf = clf.fit(X_train_vectorized,y_train)
preds2 = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds2))

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       632
           1       0.84      0.93      0.88       118

    accuracy                           0.96       750
   macro avg       0.91      0.95      0.93       750
weighted avg       0.96      0.96      0.96       750



In [118]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_vectorized,y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))
.get_params()

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       632
           1       0.97      0.97      0.97       118

    accuracy                           0.99       750
   macro avg       0.98      0.98      0.98       750
weighted avg       0.99      0.99      0.99       750



In [ ]:
logreg = LogisticRegression().fit(X_train_vectorized, y_train)
preds = logreg.predict(X_test_vectorized)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       616
           1       0.97      0.93      0.95       134

    accuracy                           0.98       750
   macro avg       0.98      0.96      0.97       750
weighted avg       0.98      0.98      0.98       750

